In [1]:
import pandas as pd

In [2]:
# Load the data
train_data = pd.read_csv("../data/cache/training_unwound.csv")
validation_data = pd.read_csv("../data/cache/validation_unwound.csv")
train_data["language"] = train_data["language"].astype("category")
validation_data["language"] = validation_data["language"].astype("category")
train_data.head()

,row_index,node,is_root,language,tree_diameter,tree_size,tree_edges,number_of_centroids,average_degree,number_of_leaves,...,harmonic_centrality,betweenness_centrality,pagerank,katz_centrality,current_flow_closeness,current_flow_betweenness,load_centrality,percolation_centrality,second_order_centrality,laplacian_centrality
0,0,6,False,Japanese,14,23,22,1,1.913043,6,...,5.823846,0.090909,0.048565,0.209086,0.007246,0.090909,0.090909,0.090909,98.762341,0.101449
1,0,4,False,Japanese,14,23,22,1,1.913043,6,...,4.561122,0.000000,0.027162,0.188298,0.006289,0.000000,0.000000,0.000000,112.481110,0.043478
2,0,2,False,Japanese,14,23,22,1,1.913043,6,...,6.991703,0.255411,0.066901,0.228660,0.008403,0.255411,0.255411,0.255411,84.451169,0.159420
3,0,23,False,Japanese,14,23,22,1,1.913043,6,...,5.157179,0.000000,0.025477,0.190256,0.007143,0.000000,0.000000,0.000000,100.149888,0.057971
4,0,20,False,Japanese,14,23,22,1,1.913043,6,...,7.146825,0.311688,0.042552,0.213357,0.009615,0.311688,0.311688,0.311688,71.147734,0.130435


In this script, we'll fit a MLP model to the dataset to predict whether the node is the root or not.

In [3]:
def one_hot_encode(df: pd.DataFrame) -> pd.DataFrame:
    """
    One-hot encode the 'language' column in the DataFrame.
    """
    return pd.get_dummies(df, columns=["language"], prefix="", prefix_sep="", drop_first=False)

We want to find the best hyperparameters for the MLP model. To do this, we will use a grid search with 5-fold cross-validation.

The search space, however, is quite large, so this will need to be done in the background. Thus, take a look a the `run_mlp.py` script to see the code, which was developed to be able to run it with `nohup` in the background.

Now let's try to get its generalization performance with the single-split validation we did

In [ ]:
from collections import defaultdict
from sklearn.metrics import accuracy_score

In [ ]:
X_val = one_hot_encode(validation_data.drop(columns=["row_index", "node", "is_root"]))
y_val = validation_data["is_root"]

In [ ]:
predictions = model.predict(X_val)
print(f"Node-based accuracy: {accuracy_score(y_val, predictions.reshape(-1) > 0.5):.2f}")

1247/1247 ━━━━━━━━━━━━━━━━━━━━ 1s 665us/step
Node-based accuracy: 0.70


In [ ]:
sentence_predictions = defaultdict(dict)
sentence_real_root = {}
for (_, row), probs in zip(validation_data.iterrows(), predictions):
    sentence_predictions[row["row_index"]][int(row["node"])] = probs[0]
    if row["is_root"]:
        sentence_real_root[row["row_index"]] = row["node"]

if not set(sentence_predictions.keys()) == set(sentence_real_root.keys()):
    raise ValueError("Mismatch between sentence predictions and real roots.")


def get_predicted_root(row: pd.Series) -> str:
    """
    Get the predicted root node for a sentence.
    """
    sentence_id = row.name
    probs = sentence_predictions[sentence_id]
    return max(probs.keys(), key=probs.get)


validation_prediction = pd.DataFrame.from_dict(sentence_real_root, orient="index", columns=["root"])
validation_prediction["predicted_root"] = validation_prediction.apply(get_predicted_root, axis=1)
print(
    f"Sentence-based accuracy: {accuracy_score(validation_prediction['root'], validation_prediction['predicted_root']):.2f}"
)

Sentence-based accuracy: 0.26
